In [2]:
import pandas as pd
import numpy as np
import scipy.io
import math
from collections import Counter
import copy
import warnings
import tensorflow as tf
from tensorflow.keras.layers import *
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import BatchNormalization
warnings.filterwarnings("ignore")

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [3]:

def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):

            pretty(value, indent + 2)
            print('-----------------------')
        else:
            print('\t' * (indent + 2) + str(value))
            print(" ")
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
def extractSubject(name):
    ex1Path = 'DB1/' + name + '/' + name + '_A1_E1.mat'
    print(ex1Path)
    ex1 = scipy.io.loadmat(ex1Path)
    emg = ex1['emg']
    EMGdf = pd.DataFrame.from_dict(emg)
    stimulus = ex1['stimulus']

    ex2Path = 'DB1/' + name + '/' + name + '_A1_E2.mat'
    ex2 = scipy.io.loadmat(ex2Path)
    emg2 = ex2['emg']
    EMGdf2 = pd.DataFrame.from_dict(emg2)
    stimulus2 = ex2['stimulus']

    ex3Path = 'DB1/' + name + '/' + name + '_A1_E3.mat'
    ex3 = scipy.io.loadmat(ex3Path)
    emg3 = ex3['emg']
    EMGdf3 = pd.DataFrame.from_dict(emg3)
    stimulus3 = ex3['stimulus']

    Movements = {}
    for m in range(1, 53):
        if (m < 13):
            movementIndices = np.where(stimulus == m)[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf
        elif (m < 30):
            movementIndices = np.where(stimulus2 == (m - 12))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf2
        else:
            movementIndices = np.where(stimulus3 == (m - 29))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf3

        Repetitions = {}
        for r in range(1, 7):
            startIndex = repetitions[r - 1][0]
            LastIndex = repetitions[r - 1][len(repetitions[r - 1]) - 1]
            df = EMG.iloc[startIndex:LastIndex, 0:10]
            df.reset_index(drop=True, inplace=True)
            Repetitions["R{0}".format(r)] = df
        Movements["Movement{0}".format(m)] = Repetitions
    return Movements
def Average(lst):
    return sum(lst) / len(lst)


In [4]:

columns = {'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'Train','Movement'}
X_train = []
y_train = []
X_test = []
y_test = []
for s in range(1,2):
    subject = 'S' + str(s)
    dff = pd.DataFrame.from_dict(extractSubject(subject))
    for m in range(1,53):
        M = dff['Movement'+str(m)]
        for r in range(1,7): 
            r_index = "R" + str(r)
            rep = M[r_index]
            if (r in [1, 3, 4, 6]):
                train = True
            else:
                train = False
            i=0    
            for x in range(0, len(rep)):
                entry = rep.iloc[x, :]
                entry.columns = {'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10'}
                entry.reset_index(drop=True, inplace=True)
                matrix = entry.to_numpy()
                if(train):
                    X_train.append(matrix)
                    y_train.append(m)
                else :
                    X_test.append(matrix)
                    y_test.append(m)  
                print(i)    
                i+=1
                
                       


DB1/S1/S1_A1_E1.mat
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
27

In [5]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
print("X Train shape: ", X_train.shape)
print("X Test shape: ", X_test.shape)
print("Y Train shape: ", y_train.shape)
print('------------------')
X_train = np.array(X_train).reshape(X_train.shape[0],1,10)
X_test = np.array(X_test).reshape(X_test.shape[0],1,10)
y_train = np.array(y_train).astype('float32').reshape(X_train.shape[0],1)
y_test = np.array(y_test).astype('float32').reshape(X_test.shape[0],1)

y_train= tf.one_hot(y_train,52)
y_test= tf.one_hot(y_test,52)
y_test = np.array(y_test).astype('float32').reshape(X_test.shape[0],52)
y_train = np.array(y_train).astype('float32').reshape(X_train.shape[0],52)

print("X Train shape: ", X_train.shape)
print("X Test shape: ", X_test.shape)
print("Y Train shape: ", y_train.shape)

X Train shape:  (107344, 10)
X Test shape:  (53694, 10)
Y Train shape:  (107344,)
------------------
X Train shape:  (107344, 1, 10)
X Test shape:  (53694, 1, 10)
Y Train shape:  (107344, 52)


In [17]:
input = Input(shape =(1,10,1))

X = BatchNormalization(input)
X = Conv2D(64, (1, 1), padding='valid', kernel_initializer='glorot_normal')(input)
X = Activation('relu', name='b1_relu')(X)

X = BatchNormalization(input)
X = Conv2D(64, (3, 3), padding='valid', kernel_initializer='glorot_normal')(X)
X = Activation('relu', name='b2_relu')(X)

X = Dense(64, activation='relu', name='first_layer')(X)
X = Dense(512, activation='relu', name='first_layer')(X)
X = Dropout(0.15, name='b4_dropout')(X)
X = Dense(512, activation='relu', name='first_layer')(X)
X = Dropout(0.15, name='b4_dropout')(X)
X = Dense(128, activation='relu', name='first_layer')(X)
X = Dropout(0.15, name='b4_dropout')(X)
X = Dense(52, padding='same',  kernel_initializer='glorot_normal')(X)
X = Activation('softmax', name='b5_soft')(X)

model = Model(input, X)
model.compile(optimizer=SGD(learning_rate=0.05), batch_size=1000, loss="categorical_crossentropy", metrics=['accuracy'])
# print(X_train.shape)

model.fit(X_train, y_train, epochs=28, validation_data=(X_test, y_test))
print(model.evaluate(X_test, y_test))
model.summary()


TypeError: Expected an int or a list/tuple of ints for the argument 'axis', but received: <KerasTensor: shape=(None, 1, 10, 1) dtype=float32 (created by layer 'input_12')>

In [239]:
# model.summary()
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
class_labels = np.argmax(y_pred, axis=1) 
y_df = pd.DataFrame(class_labels)
y_df.to_csv('y_pred.csv')
class_labels_2 = np.argmax(y_test, axis=1) 
y_df_2 = pd.DataFrame(class_labels_2)
y_df_2.to_csv('y_test.csv')
print(class_labels)
print(class_labels_2)
y_test_new = [most_frequent(class_labels_2[x:x + 83]) for x in range(0, len(class_labels_2), 83)]
y_predicted_new = [most_frequent(class_labels[x:x + 83]) for x in range(0, len(class_labels), 83)]
accuracy_modified = accuracy_score(y_test_new, y_predicted_new)
print(accuracy_modified)


[10 10 10 ... 36 36 36]
[1 1 1 ... 0 0 0]
0.36117936117936117


In [240]:
y_predicted_new
# y_train.shape
# y_test.shape

[10,
 10,
 10,
 10,
 3,
 10,
 10,
 10,
 10,
 7,
 10,
 10,
 10,
 10,
 7,
 8,
 10,
 36,
 10,
 10,
 11,
 10,
 2,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 36,
 10,
 18,
 36,
 3,
 10,
 20,
 20,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 2,
 25,
 25,
 25,
 10,
 36,
 10,
 36,
 36,
 28,
 10,
 10,
 10,
 10,
 36,
 36,
 27,
 10,
 41,
 10,
 10,
 10,
 50,
 47,
 36,
 12,
 37,
 37,
 10,
 10,
 10,
 10,
 10,
 51,
 41,
 41,
 8,
 41,
 43,
 43,
 44,
 44,
 8,
 44,
 8,
 7,
 47,
 47,
 48,
 48,
 2,
 38,
 49,
 50,
 50,
 51,
 51,
 29,
 3,
 1,
 1,
 2,
 2,
 3,
 3,
 10,
 3,
 4,
 3,
 10,
 9,
 7,
 7,
 8,
 8,
 9,
 9,
 10,
 17,
 11,
 9,
 17,
 12,
 13,
 13,
 10,
 43,
 10,
 10,
 17,
 17,
 17,
 17,
 18,
 18,
 19,
 19,
 20,
 17,
 51,
 11,
 17,
 7,
 13,
 1,
 24,
 24,
 25,
 28,
 26,
 13,
 41,
 17,
 28,
 28,
 28,
 10,
 6,
 34,
 36,
 3,
 3,
 36,
 33,
 27,
 34,
 17,
 3,
 35,
 8,
 17,
 37,
 3,
 10,
 10,
 36,
 36,
 40,
 10,
 27,
 41,
 3,
 40,
 43,
 43,
 3,
 17,
 3,
 41,
 36,
 3,
 17,
 44,
 8,
 36,
 27,
 49,
 7,
 41,
 41,
 11,
 51,

In [ ]:
model.history['accuracy']
model.history['loss']